In [ ]:
import pandas as pd
from pyproj import Proj, transform
from pulp import *
from haversine import haversine

# Google Drive 연동
from google.colab import drive
drive.mount('/content/drive')

# 데이터 불러오기
infra_df = pd.read_csv('/content/drive/MyDrive/mclp/인프라.csv', encoding='utf-8')
accessibility_df = pd.read_csv('/content/drive/MyDrive/mclp/접근성.csv', encoding='utf-8')
demand_df = pd.read_csv('/content/drive/MyDrive/mclp/수요.csv', encoding='utf-8')

# 인프라 데이터의 좌표를 4326으로 변환
in_proj = Proj(init='epsg:5179')
out_proj = Proj(init='epsg:4326')

infra_df['경도'], infra_df['위도'] = transform(in_proj, out_proj, infra_df['xcoord'].values, infra_df['ycoord'].values)

# 수요 데이터의 좌표를 4326으로 변환
demand_df['경도'], demand_df['위도'] = transform(in_proj, out_proj, demand_df['xcoord'].values, demand_df['ycoord'].values)

# MCLP 모델링
model = LpProblem(name="Maximum_Coverage_Location_Problem", sense=LpMaximize)

# 좌표를 저장할 변수
x = LpVariable.dicts("location", infra_df.index, 0, 1, LpBinary)

# 목적함수 설정
model += lpSum([demand_df['demand'][i] * x[i] for i in infra_df.index])

# 수요지수, 접근성지수, 인프라지수에 대한 제약 설정
for i in infra_df.index:
    model += lpSum([haversine((infra_df['위도'][i], infra_df['경도'][i]),
                              (infra_df['위도'][j], infra_df['경도'][j])) * x[j] for j in infra_df.index]) <= 250

# 최적화 수행
model.solve()

# 결과 확인
result_locations = [i for i in infra_df.index if x[i].value() == 1]

# 결과값을 데이터프레임에 추가하고 수요지수로 정렬하여 순위 매기기
result_df = infra_df.loc[result_locations, ['위도', '경도']]

# 상위 10곳만 선택
result_top10 = result_df.head(10)

# 상위 10곳의 순위 정보를 추가하여 CSV 파일로 저장
result_top10['순위'] = range(1, 11)
result_top10.to_csv('/content/drive/MyDrive/mclp/최적입지_결과_순위.csv', index=False)
